<a href="https://colab.research.google.com/github/AntonioPL/BCN_Noise/blob/main/Unsupervised_Learning_Noise_Pollution_Gegraphic_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install pyproj
!pip install pyproj

     |████████████████████████████████| 6.6MB 5.1MB/s 


In [3]:
# Common parameters

ClusterColors=['black','pink','lightblue','beige','darkblue']
NoiseBandColors=['#238443','#78C649','#C2E699','#FFFFB2','#FECC5C','#FD8D3C','#FF0909','#B30622','#67033B','#1C0054']


In [4]:
## Latitude and Longitude data transformation
import pyproj as proj
import warnings
warnings.filterwarnings("ignore")

# Proyections configuration
pj_target = proj.Proj(init='epsg:4326') # objective proyection
pj_data  = proj.Proj(init='epsg:25830') # data proyection

x1= 726882.491
y1= 4370748.231
lon1, lat1 = proj.transform(pj_data, pj_target , x1,y1)

x2= 727265.727
y2= 4371653.468
lon2, lat2 = proj.transform(pj_data, pj_target , x2,y2)

In [5]:
import pandas as pd
# Load nodes data
data = pd.read_csv("/content/drive/My Drive/LatitudLongitud.csv",delimiter = ";",decimal=",", encoding='utf8')
# show first and last 5 instances
print(data.head(5))
data.tail(5)

                     ID        Direccion  ...    Latitud  Longitud
0             Alella 45      C_Alella_45  ...  41.431418  2.177110
1  Av. Parallel - AP-06        C_Manso_1  ...  41.375478  2.158631
2   Av. Parallel - AP18   Av_Parallel_81  ...  41.374750  2.166537
3   Av. Parallel - AP23  Av_Parallel_115  ...  41.374754  2.161176
4           Bac de Roda    C_Tortella_20  ...  41.403457  2.209004

[5 rows x 6 columns]


,ID,Direccion,fichero,codigo,Latitud,Longitud
66,Ulldecona II,C_Ulldecona_27,Ulldecona II,BCN_67,41.350755,2.141211
67,Ulldecona,C_Ulldecona_77,Ulldecona,BCN_68,41.353070,2.143779
68,Urgell - Arago,C_Comte d'Urgell_126,Urgell - Aragó,BCN_69,41.384720,2.155510
69,Urgell - Diputacio,C_Comte d'Urgell_88,Urgell - Diputació,BCN_70,41.383016,2.157764
70,Xifre,C_Xifre_94,Xifré,BCN_71,41.411052,2.180858


In [6]:
# Load folium 
import folium

# Initialise map
m = folium.Map(location=[41.4, 2.17], zoom_start=12.4)

# Introduce data in map
for a, b, c, d, e in zip(data['Latitud'],data['Longitud'],data['codigo'],data['ID'],data['Direccion']): 
  folium.Marker([a, b], tooltip="Node {} is called {} and it's located in {}".format(c,d,e)).add_to(m) 

In [7]:
# Visualise map
m

In [8]:
import numpy as np
# Load noise data
data2 = pd.read_csv("/content/drive/My Drive/Asignaciones 4 clusters.csv",delimiter = ";",decimal=",", encoding='utf8')
data2.columns = ['Codigo', 'Ld', 'Le', 'Ln','S', 'CLUSTER']
data2['Lden']=10*np.log10((12*(10**(data2['Ld']/10.0))+4*(10**((data2['Le']+5)/10.0))+8*(10**((data2['Ln']+10)/10.0)))/24.0)
print(data2.head(5))
data3=data2.merge(data,left_on="Codigo", right_on="codigo")
data3.head(5)

   Codigo         Ld         Le         Ln         S  CLUSTER       Lden
0   BCN_1  65.874984  66.018174  62.460497  1.689935        2  69.962921
1  BCN_10  63.033470  66.334262  60.438055  2.739746        2  68.425170
2  BCN_11  60.424388  59.716755  56.720852  3.032346        4  64.169261
3  BCN_12  62.902817  61.531414  57.138029  2.103326        4  65.378198
4  BCN_13  55.689005  60.437400  56.127484  2.851353        4  63.293462


,Codigo,Ld,Le,Ln,S,CLUSTER,Lden,ID,Direccion,fichero,codigo,Latitud,Longitud
0,BCN_1,65.874984,66.018174,62.460497,1.689935,2,69.962921,Alella 45,C_Alella_45,Alella 45,BCN_1,41.431418,2.177110
1,BCN_10,63.033470,66.334262,60.438055,2.739746,2,68.425170,Carders 36,C_Carders_36,Carders 36,BCN_10,41.387034,2.180464
2,BCN_11,60.424388,59.716755,56.720852,3.032346,4,64.169261,Carrer Fluvi<e0> 120,C_Fluvia_129,Carrer Fluvià 120,BCN_11,41.409598,2.203124
3,BCN_12,62.902817,61.531414,57.138029,2.103326,4,65.378198,Castella 1,C_Castella_1,Castella 1,BCN_12,41.404468,2.198824
4,BCN_13,55.689005,60.437400,56.127484,2.851353,4,63.293462,Coberta de Sants 1,C_Burgos_22,Coberta de Sants 1,BCN_13,41.372082,2.133523


In [9]:
# Cluster Map
# Initialise map
m2 = folium.Map(location=[41.4, 2.17], zoom_start=12.4)

# Introduce data in map
for a, b, c, d, e, f in zip(data3['Latitud'],data3['Longitud'],data3['codigo'],data3['ID'],data3['Direccion'],data3['CLUSTER']): 
  color=ClusterColors[f-1]
  folium.Marker([a, b], tooltip="Node {} is called {} and it's located in {}".format(c,d,e),icon=folium.Icon(color=color,icon="signal")).add_to(m2) 

In [10]:
# Visualise map2
m2

In [11]:
# Lden Map
# Initialise map
m3 = folium.Map(location=[41.4, 2.17], zoom_start=12.4)

# Introduce data in map
for a, b, c, d, e, f, g in zip(data3['Latitud'],data3['Longitud'],data3['codigo'],data3['ID'],data3['Direccion'],data3['CLUSTER'],data3['Lden']): 
  color=ClusterColors[f-1]
  if(g<40):
    color2 = NoiseBandColors[0]
  elif(g<45):
    color2 = NoiseBandColors[1]
  elif(g<50):
    color2 = NoiseBandColors[2]
  elif(g<55):
    color2 = NoiseBandColors[3]
  elif(g<60):
    color2 = NoiseBandColors[4]
  elif(g<65):
    color2 = NoiseBandColors[5]
  elif(g<70):
    color2 = NoiseBandColors[6]
  elif(g<75):
    color2 = NoiseBandColors[7]
  elif(g<80):
    color2 = NoiseBandColors[8]
  elif(g>=80):
    color2 = NoiseBandColors[9]
  else:
    color2 = 'pink' 
  folium.Marker([a, b], tooltip="Node {} is called {} and it's located in {}".format(c,d,e),icon=folium.Icon(color='white',icon='signal',icon_color=color2)).add_to(m3) 

In [12]:
# Visualise map3
m3

In [13]:
# Lden and cluster map
# Initialise map
m4 = folium.Map(location=[41.4, 2.17], zoom_start=12.4)

# Introduce data in map
for a, b, c, d, e, f, g in zip(data3['Latitud'],data3['Longitud'],data3['codigo'],data3['ID'],data3['Direccion'],data3['CLUSTER'],data3['Lden']): 
  color=ClusterColors[f-1]
  if(g<40):
    color2 = NoiseBandColors[0]
  elif(g<45):
    color2 = NoiseBandColors[1]
  elif(g<50):
    color2 = NoiseBandColors[2]
  elif(g<55):
    color2 = NoiseBandColors[3]
  elif(g<60):
    color2 = NoiseBandColors[4]
  elif(g<65):
    color2 = NoiseBandColors[5]
  elif(g<70):
    color2 = NoiseBandColors[6]
  elif(g<75):
    color2 = NoiseBandColors[7]
  elif(g<80):
    color2 = NoiseBandColors[8]
  elif(g>=80):
    color2 = NoiseBandColors[9]
  else:
    color2 = 'pink'   
  folium.Marker([a, b], tooltip="Node {} is called {} and it's located in {}".format(c,d,e),icon=folium.Icon(color=color,icon='signal',icon_color=color2)).add_to(m4) 

In [14]:
# Visualise map4 Lday
m4